# f1_extract_sentences_with_entities
From the file sentences.txt extract all the sentence who contain entities and write them into .txt file

In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
import pandas as pd
import requests
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import MWETokenizer
import import_ipynb
import c1_extract_paragraphe_issue as c1 
import c2_extract_sentence_issue as c2 
import g1_generate_dictionary as g1
import collections
from collections import Counter
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

importing Jupyter notebook from c1_extract_paragraphe_issue.ipynb
Issue  34
importing Jupyter notebook from c2_extract_sentence_issue.ipynb
importing Jupyter notebook from g1_generate_dictionary.ipynb
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tatianacogne/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def write_list_collected_issue_org(data, issue_number):
    """Write list_sentence_with_country.txt file. """
    s = "Text/original_"+str(issue_number)+".csv"
    with open(s, 'w', newline='') as file:
        writer = csv.writer(file)
        #header
        #writer.writerow(('issue_number','country 1','country 2','behalf','support','spokewith','agreement','delay','opposition','criticism','cooperation','sentence'))
        #list
        writer.writerows(data)

In [3]:
def write_list_collected_issue(list_with_two_entities, number,generated):
    if(generated):
        s = "Text/interactions_"+str(number)+".csv"
    """ Generate list_meetings.csv file."""
    with open(s, 'w', newline='') as file:
        writer = csv.writer(file)
        #header
        writer.writerow(('issue_number','country 1','country 2','behalf','support','spokewith','agreement','delay','opposition','criticism','cooperation','sentence'))
        #list
        writer.writerows(list_with_two_entities)

In [4]:
def extract_from_txt_sentences(txt_file):
    """ Open file that contains all the sentences."""
    list_tp = open(txt_file)
    return list_tp

In [5]:
def clean_tp(sentence):
    """ Clean the sentence by removing special char."""
    s = sentence.replace("\r\n\s\s+"," ")
    s = s.replace("\r\n"," ")
    s = s.replace('\t','')
    s = s.replace("\s\s+"," ")
    s = s.replace("\\."," ")
    s = s.replace("\\r\\n"," ")
    p = re.compile(r'<.*?>')
    return p.sub('', s)

In [6]:
def extract_s2e_issue():
    """ Extract all the sentence with at least one party inside. """

    #List sentences
    #paragraphes = c1.extract_paragraphes_from_issue(number)
    sentences = c2.extract_from_txt_sentences(open("Text/sentences_s1e.txt"))
    sentences = list(set(sentences))[20000:20100]
    list_entities = [s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]
    #Create list that wil contain all the sentences with at least two entities
    sentences_s2e = []
    for i, s in enumerate(sentences):
        print(f'{(i+1)/len(sentences)*100:.2f}%', end='\r')
    #Split line into words with tokenizer to detetc entity
    
        tokens = tokenize_sentence(s)
        done = False
        for e1 in list_entities: 
            for e2 in list_entities:
                if(set([e1,e2]).issubset(set(tokens)) and e1 != e2):
                    sentences_s2e.append(s)
                    done = True
                    break
            if done:
                break
    
    s = "Text/sentences_s2e.txt"
    write_list_collected(sentences_s2e,s)
    return sentences_s2e

In [7]:
def extract_s1e_issue():
    """ Extract all the sentence with at least one party inside. """

    #List sentences
    #paragraphes = c1.extract_paragraphes_from_issue(number)
    sentences = c2.extract_from_txt_sentences(open("Text/sentences.txt"))
    sentences = list(set(sentences))
    list_entities = [s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]
    #Create list that wil contain all the sentences with at least two entities
    sentences_s1e = []
    for i, s in enumerate(sentences):
    #Split line into words with tokenizer to detect entity
        s = s.replace('\\t','')
        s = re.sub(r'\([^)]*\)', '', s)
        tokens = tokenize_sentence(s,False)

        if(len(set(tokens).intersection(set(ENTITIES)) )> 1):
            sentences_s1e.append(s)

        print(f'{(i+1)/len(sentences)*100:.2f}%', end='\r')
    s = "Text/sentences_s1e.txt"
    issue_number = 1111
    c2.write_sentences(issue_number,sentences_s1e)
    return sentences_s1e

In [8]:
def extract_s2e_issue_number(issue_number):
    """ Extract all the sentence with at least one party inside. """

    #List sentences
    #paragraphes = c1.extract_paragraphes_from_issue(number)
    p = c1.extract_paragraphes_from_issue(issue_number)
    sentences = c2.extract_from_txt_sentences(p)
    #list_entities = [s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))]
    list_entities = [s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]
    #Create list that wil contain all the sentences with at least two entities
    sentences_s2 = []

    for i, s in enumerate(sentences):
    #Split line into words with tokenizer to detect entity
        tokens = tokenize_sentence(s,False)

        if(len(set(tokens).intersection(set(ENTITIES)) )> 1):
            sentences_s2.append(s)
    c2.write_sentences(issue_number,sentences_s2)
    return sentences_s2

# Find Interactions

### Find Party Groupings 

In [9]:
import re
def find_party_groupings(sentences):
    links = ['for the','on behalf of the ']
    list_l = []
    for s in sentences:

        l = re.findall('for the.*?,| for the.*?and',s)
        l = [s.replace('for the ','') for s in l if s.replace('for the ','')[0].isupper()]
        l = [s.replace(',','') for s in l]


        b = re.findall('on behalf of  .*?,| on behalf of the .*?said| on behalf of the .*?and',s)
        b = [s.replace('on behalf of  ','') for s in b]
        b = [s.replace(',','') for s in b]

        c = re.findall('speaking for the.*?,|speaking for the .*?said|speaking for the .*?and',s)
        c = [s.replace('speaking for  ','') for s in b]
        c = [s.replace(',','') for s in b]
        
        list_l +=  l+c+b

    write_list_collected(list(set(list_l)),"party_groupings.txt")
    x = list(set(list_l))

    return x


## Start to filter and analyse the sentences

## 1 - Define functions to pos tag the sentences

In [10]:
def tokenize_sentence(sentence, country):
    """Split the sentence in a way that the entities are together and will be able to be detected."""
    # Extract list entities
    list_entities = [s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]
    list_entities = [s.replace('class="textstory"','') for s in list_entities]
    
    tokens_entities = [l.split(' ') for l in list_entities]
    if(country):
        tokens_entities.append(['on','behalf','of'])
        tokens_entities.append(['for'])
        tokens_entities.append(['US','$'])
        tokens_entities.append(['speaking','for'])
    else :

        tokens_entities.append(['on','behalf','of','the'])
        tokens_entities.append(['spoke','with'])
        tokens_entities.append(['on','behalf','of'])
        tokens_entities.append(['for','the'])
        tokens_entities.append(['US','$'])
        tokens_entities.append(['speaking','for','the'])
    tokens_entities.append(['supported','by'])
    tokens_entities.append(['opposed','by'])
    tokenizer1 = MWETokenizer(tokens_entities, separator=' ')
    tokenizer2 = MWETokenizer([['G-77','CHINA']], separator='/')
    tokenizer3 = MWETokenizer([['G-77/',' CHINA']], separator=' ')
    if(type(sentence) == list):
        line = sentence[0].replace(",","")
    else: 
        line = sentence.replace(",","")
    line_splited = word_tokenize(line)
    tokens = tokenizer1.tokenize(line_splited) 
    tokens = tokenizer2.tokenize(tokens) 
    tokens = tokenizer3.tokenize(tokens) 
    tokens = [clean_tp(token) for token in tokens]

    return tokens

In [12]:
find_pos_tagged_s2e(['92s '])

[[('92s', 'CD')]]

In [13]:
ENTITIES = list(set([s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]))
COALITIONS = []
SUPPORTS_LINKS = ['with','and','for the','on behalf of the','supported by','speaking for the']
OPPOSITION_LINKS= ['opposed by','while','opposed by the']
LIST_TAGS = ['IN', 'CC', 'NN', 'NNP', 'JJ','NNPS','MD','VBP','VB','VBZ','VBD','RB','VBN','PRP']
PARTY_GROUPINGS = sorted(set([s.replace('\n','').upper() for s in list(open('Text/party_grouping_clean.txt'))] + [s.replace('\n','').title() for s in list(open('Text/party_grouping_clean.txt'))] + [s.replace('\n','') for s in list(open('Text/party_grouping_clean.txt'))]))
list_entities = list(set([s.replace('\n','') for s in list(open('Text/entities_interactions.txt'))] + [s.replace('\n','').title() for s in list(open('Text/entities_interactions.txt'))]))
PARTIES = sorted(set(list_entities).difference(set(PARTY_GROUPINGS)))

## 2 - Need to keep all the following tag : 'IN', 'CC', 'NN', 'NNP', 'JJ','NNPS','MD'

In [14]:
def find_patterns(pos_tagged, list_tags):  
    """ Find all the pattern in list_tags needed in sentences pos_tagged. """
    groups = [x[0] for x in pos_tagged[0] if x[1] in list_tags]
    return groups

## 3 - Define the groups : 1 (cooperation only) or two groups (opposition)

In [15]:
def find_1g(groups): 
    """ Find all the entities in the groups. Return a list of entities"""
    groups = [g.replace(',','') for g in groups]
    entities = set(groups).intersection(set(ENTITIES))
    return list(entities)

In [16]:
def find_2g(groups, opp):
    """ Find all the entities for each groups. Return two lists of entities"""

    index = groups.index(opp)

    g1 = find_1g(groups[:index])
    g2 = find_1g(groups[index +1:])

    return g1, g2 

In [17]:
def detect_groups_cooperations(groups):
    """ Return one or two groups with only entities and the original sentence. Return a list with one or two list"""

    # Case 1 : Opposition between two groups
    if(set(OPPOSITION_LINKS).intersection(set(groups)) != set()):
        opp = list(set(OPPOSITION_LINKS).intersection(set(groups)))[0]
        
        g1, g2 = find_2g(groups, opp)
        return [g1, g2]

    # Case 2 : Only support
    else:
        g1 = find_1g(groups)
        return [g1]

## 4 - Detect "on behalf of"/ "for the"/speaking for for coalition 

In [18]:
def find_coalitions(groups, sentence):
    """ Remove all the parties in groups that speak for a coalition, return list group updated if the case. """
    groups_update = []
    list_removed = []

    for group in groups: 
        set_group = set(group)
        token =tokenize_sentence(sentence,False)

        links = ['for the','on behalf of the','speaking for the','on behalf of','for']

        if(set(token).intersection(set(links)) != set()):
            for i in range(len(token)-2):

                if(token[i] in PARTIES and token[i+1] in links and token[i+2] in PARTY_GROUPINGS and token[i] in set_group):

                    l = list(set(token).intersection(set(links)))[0]
                    list_removed.append((token[i],l))
                    
                    set_group.remove(token[i])

            groups_update.append(list(set_group))
        else : 
            groups_update.append(group)
    return groups_update, list_removed 

## 5 - Find cooperation 

In [19]:
from itertools import combinations  
import itertools
def rSubset(arr, cop): 
    # return list of all subsets of length r 
    # to deal with duplicate subsets use  
    # set(list(combinations(arr, r))) 

    l = list(set(list(itertools.product(arr, arr))))

    return [(c1.upper(),c2.upper(),cop) for c1,c2 in l if c1 != c2]

In [20]:
def behalf_of(sentence, group_cooperation, link):
    token =tokenize_sentence(sentence, True)
    index = token.index(list(link)[0])
    country_A = set(token[:index]).intersection(set(group_cooperation))
    countries_B = set(group_cooperation).difference(set(country_A))
    tuples = []  
    for x in countries_B:
        tuples.append((list(country_A)[0].upper(),x,['behalf','cooperation']))
        
    tuples += rSubset(list(countries_B),['behalf of','agreement','cooperation'])

    return sorted(tuples)


In [21]:
def supported_by(sentence, group_cooperation, link ):
    token =tokenize_sentence(sentence, True)
    index = token.index(list(link)[0])

    country_A = set(token[:index]).intersection(set(group_cooperation))
    countries_B = set(group_cooperation).difference(set(country_A))
    tuples = []  

    for x in countries_B:
        
        tuples.append((x,list(country_A)[0].upper(),['support','cooperation']))
        
    tuples += rSubset(list(countries_B),['agreement','cooperation'])

    return sorted(tuples)

In [22]:
def with_c(sentence, group_cooperation, link):
    token =tokenize_sentence(sentence, True)
    index = token.index(list(link)[0])
    country_A = set(token[:index]).intersection(set(group_cooperation))

    countries_B = set(group_cooperation).difference(set(country_A))
    tuples = []  
    for x in countries_B:
        tuples.append((list(country_A)[0].upper(),x,['spokeWith','cooperation']))
        tuples.append((x,list(country_A)[0].upper(),['spokeWith','cooperation']))

    tuples += rSubset(list(countries_B), ['spokeWith','agreement','cooperation'])

    return sorted(tuples)

In [23]:
def check_link(link, sentence, group_cooperation):
    """Verify that there is entities in both sides of the link"""
    token =tokenize_sentence(sentence, True)

    index = token.index(list(link)[0])
    country_A = set(token[:index]).intersection(set(group_cooperation))
    return country_A != set()


In [24]:
def coop(sentence, group_cooperation):
    """group_cooperation i one list """
    token =tokenize_sentence(sentence, True)

    behalf = ['speaking for','on behalf of']
    behalf = set(token).intersection(set(behalf))

    support = ['supported by','supported by the']

    support = set(token).intersection(set(support))


    tuples = []
    cooperation = []
    if(behalf != set()):
        return behalf_of(sentence, group_cooperation, behalf)
    if(support != set() and  check_link(support, sentence, group_cooperation)):
        return supported_by(sentence, group_cooperation, support)

    else :
        return rSubset(group_cooperation,['agreement','cooperation'])


In [25]:
import itertools
def opposed_by(sentence, group_cooperation, link):
    
    token =tokenize_sentence(sentence, True)
    index = token.index(list(link)[0])
    countries_A = group_cooperation[0]
    countries_B = group_cooperation[1]

    # Create tuples for the opposition
    tuples = list(itertools.product(countries_A, countries_B))

    tuples = [(c2.upper(),c1.upper(),['opposition']) for c1,c2 in tuples] 



    #Add cooperation between both groups
    splited = sentence.split(list(link)[0])
    if(len(countries_A)!=1):
        tuples_A = coop([splited[0]], countries_A)
        tuples += tuples_A
    
    if(len(countries_B)!=1):
        tuples_B = coop([splited[1]], countries_B)
        tuples += tuples_B
    
    return tuples


In [26]:
def criticized_by(sentence, group_cooperation, link):
    token =tokenize_sentence(sentence, True)
    index = token.index(list(link)[0])
    countries_A = list(set(group_cooperation[0]).intersection(set(token[:index])))
    countries_B = list(set(group_cooperation[0]).intersection(set(token[index+1:])))
    
    # Create tuples for the opposition
    tuples = list(itertools.product(countries_A, countries_B))
    tuples = [(c1.upper(),c2.upper(),['criticism']) for c1,c2 in tuples]

    #Add cooperation between both groups
    splited = sentence[0].split(list(link)[0])
    
    if(len(countries_A)!=1):
        tuples_A = coop(splited[0], countries_A)
        tuples += tuples_A
    
    if(len(countries_B)!=1):
        tuples_B = coop(splited[1], countries_B)
        tuples += tuples_B
    
    return tuples    

In [27]:
def write_line(issue_number, x):
    """  'behalf'	 'support'	 'spokewith'	 'agreement'	 'delay'	 'opposition'	 'criticism'	 'cooperation'"""
    dict_interactions = {'behalf' : 0, 'support' : 0, 'agreement':0 , 'opposition':0,'criticism' :0, 'cooperation':0}
    for int in x[2]:
        dict_interactions[int] = 1
    values = list(dict_interactions.values())

    v = []
    v.append(issue_number)
    v.append(x[0])
    v.append(x[1])
    v += values
    v.append(x[3])
    return v
    


In [28]:
def remove_double_s(sentences):

    tags_wanted = ['VBD','MD']
    words_wanted = ['and']
    s2 = []
    sentences_filtered = []
    s_to_filter = []
    s_filtered = []
    for s in sentences:
        pos_tagged = find_pos_tagged_s2e([s])
        
        filtered = [x for x in pos_tagged[0] if x[0] in PARTIES or x[0] in words_wanted or x[1] in tags_wanted]

        if(len(filtered)>= 5):
            for i in range(len(filtered)-4):
                if( filtered[i][0] in PARTIES and filtered[i+1][1] in tags_wanted and filtered[i+2][0] == 'and' and  filtered[i+3][0] in PARTIES and  filtered[i+4][1]in tags_wanted):

                    s_to_filter.append((s,filtered[i+1]))
                else : 
                    s_filtered.append(s)

        else:
            s_filtered.append(s)
        
    for s in s_to_filter:

        index = s[0].index(s[1][0])
        s1 = s[0][:index]
        s2 = s[0][index+1:]
        s_filtered.append(s1)
        s_filtered.append(s2)

    
    return list(set(s_filtered))
     


In [29]:
def remove_on_99s_programme(sentences):

    set_sentences = set(sentences)
    s_to_filter = []
    s_filtered = []
    for s in sentences:

        pos_tagged = find_pos_tagged_s2e([s])[0]
      
        for i in range(len(pos_tagged)-1):
   
            if(pos_tagged[i][0] == 'on' and pos_tagged[i+1][0] in ENTITIES):

                s_to_filter.append((s,pos_tagged[i+1][0]))
                set_sentences.remove(s)

            if((pos_tagged[i+1][0] == '92s' or pos_tagged[i+1][0] == 'Programme') and pos_tagged[i][0] in ENTITIES):
                s_to_filter.append((s,pos_tagged[i][0]))
                set_sentences.remove(s)
    
    for s in s_to_filter:

        s_f = s[0].replace(s[1],'')
        s_filtered.append(s_f)

 
    s_filtered += list(set_sentences)
    
    return list(set(s_filtered))


In [30]:
def check_interaction(group_cooperation):
    if(len(group_cooperation) == 2 or len(group_cooperation[0])>1):
        return True
    else:
        return False

In [31]:
def check_doubles(sentence, group_cooperation):

    gc_new = []
    
    for g in group_cooperation:

        g_upper = [e.upper() for e in g]
        g_title = [e.title() for e in g]
        if(len(set(g_upper)) != len(g)):
            gc_new.append(list(set(g).difference(set(g_title))))
            removed = list(set(g).intersection(set(g_title)))
            for r in removed:
                sentence = sentence.replace(r,'')
        else:
            gc_new.append(g)
    return gc_new, sentence

In [32]:
def find_inversions(sentences):

    set_sentences = set(sentences)
    s_to_filter = []
    s_filtered = []
    verbs_int =['Supported','Opposed']
    for s in sentences:

        pos_tagged = find_pos_tagged_s2e([s])[0]

        for i in range(len(pos_tagged)-3):

            if(pos_tagged[i][0] in verbs_int and pos_tagged[i+1][0] == 'by' and pos_tagged[i+2][0] in ENTITIES and pos_tagged[i+3][0] in ENTITIES):

                s_to_filter.append((s,pos_tagged[i][0],pos_tagged[i+1][0],pos_tagged[i+2][0],pos_tagged[i+3][0]))
                set_sentences.remove(s)
        ## The EU ...
        for i in range(len(pos_tagged)-4):

            if(pos_tagged[i][0] in verbs_int and pos_tagged[i+1][0] == 'by' and pos_tagged[i+2][0] == 'the' and pos_tagged[i+3][0] in ENTITIES and pos_tagged[i+4][0] in ENTITIES):

                s_to_filter.append((s,pos_tagged[i][0],pos_tagged[i+1][0],pos_tagged[i+3][0],pos_tagged[i+4][0]))
                set_sentences.remove(s)
        
    
    for s in s_to_filter:
        sentence = s[0]
        verb = s[1]
        by = s[2]
        c_a = s[3]
        c_b = s[4]
        s_f =sentence.replace(by,'').replace(c_a,'').replace(c_b,'').replace(verb,'')
        s_f = c_b + " "+ verb.lower() + " " + by + " " + c_a + " " + s_f
        s_filtered.append(s_f)

 
    s_filtered += list(set_sentences)
    
    return list(set(s_filtered))


In [33]:
def get_key(val): 

    for key, value in DICTIONARY.items(): 
        if (len(value) == 1 and val == value[0]): 
             key_country = key 
        if(len(value) > 1 and val in value):
            key_country = key

            
    return DICTIONARY_NUM[key_country]

In [34]:
def add_id(interactions_):

    for i in interactions_:
        c_a = i[1]


        c_b = i[2]

        id_cb = 9999
        id_ca = 9999
        if(c_a in NAMES):
            id_ca = get_key(c_a)
        if(c_a.upper() in NAMES):
            
            id_ca = get_key(c_a.upper())
        if(c_b in NAMES):

            id_cb = get_key(c_b)
        if(c_b.upper() in NAMES):

            id_cb = get_key(c_b.upper())

        i.append(id_ca)
        i.append(id_cb)

    return interactions_

In [35]:
import pandas as pd

In [36]:
def create_df(cooperations):
    id_ca = [x[10] for x in cooperations]
    id_cb = [x[11] for x in cooperations]
    behalf = [x[3] for x in cooperations]
    support = [x[4] for x in cooperations]
    agreement =[x[5] for x in cooperations]
    opposition =[x[6] for x in cooperations]
    criticism =[x[7] for x in cooperations]
    cooperation =[x[8] for x in cooperations]
    dict_issue = {'type': 'generated','issue': cooperations[0][0],'id_ca':id_ca,'id_cb':id_cb, 'behalf':behalf,'support':support,'agreement':agreement,'opposition':opposition,'criticism':criticism,'cooperation':cooperation}
    df = pd.DataFrame(dict_issue)
    return df

In [37]:
def interactions(sentences, issue_number):
    interactions_ = []
    sentences_int = []
    for sentence_original in sentences:

        sentence_original = re.sub('class="ENB-Body" align="justify"','',sentence_original)
        sentence_original = re.sub('>','',sentence_original)
        
        pos_tagged = find_pos_tagged_s2e([sentence_original])

        tags_filtered = find_patterns(pos_tagged, LIST_TAGS)

        group_cooperation = detect_groups_cooperations(tags_filtered)
        group_cooperation, list_removed  = find_coalitions(group_cooperation, sentence_original)
            
        interactions = []
        u = ''

        sentence = sentence_original
        for c in list_removed:
            sentence = u.join(sentence.split(c[0]))
            sentence = u.join(sentence.split(c[1]))

        # Remove double (CROATIA, Croatia)
        group_cooperation, sentence = check_doubles(sentence, group_cooperation)

        # Check if interactions after all the filters
        interaction_bool = check_interaction(group_cooperation)

        if(interaction_bool):
            sentences_int.append(sentence_original)

            token = tokenize_sentence(sentence, True)
            opposition = ['opposed by','while','opposed by the']
            opposition = set(token).intersection(set(opposition))

            criticism = ['criticized']
            criticism= set(token).intersection(set(criticism))
            
            cooperations = []

            if(opposition != set() and len(group_cooperation) ==2):
                    # group_cooperation contain 2 lists

                interactions += opposed_by(sentence, group_cooperation, opposition)

            else:
                if(criticism != set()):
                        # group_cooperation contain 1 list ?
                    interactions += criticized_by(sentence, group_cooperation, criticism)
                            
                else :
        
                        # group_cooperation contain 1 list
                    interactions += coop(sentence, group_cooperation[0])
            s = str(sentence_original)        
            interactions = [(x[0],x[1],x[2],s) for x in interactions]

            interactions_ += interactions
        
    interactions_ = [ write_line(issue_number, x) for x in interactions_]
    interactions_ = add_id(interactions_)
    print('Number of observations : ', len(interactions_))
    print('Number of sentences : ', len(sentences_int))
    #c2.write_sentences(issue_number,sentences_int)
    write_list_collected_issue(interactions_ ,issue_number, True)
    df = create_df(interactions_)
    return df

In [38]:
issue_number = 255
sentences = extract_s2e_issue_number(issue_number)

https://enb.iisd.org/vol12/enb12255e.html
Issue  255


In [40]:
DICTIONARY = g1.compute_dictionary()
n = 0
DICTIONARY_NUM = dict()
for k in DICTIONARY:

    DICTIONARY_NUM[k] = n 
    n +=1
NAMES = []
for x in DICTIONARY.values():
    if(len(x) == 1):
        NAMES.append(x[0])

    else:
        NAMES += [k for k in x]

n = 0
DICTIONARY_NUM = dict()
for k in DICTIONARY:

    DICTIONARY_NUM[k] = n 
    n +=1
NAMES = []
for x in DICTIONARY.values():
    if(len(x) == 1):
        NAMES.append(x[0])

    else:
        NAMES += [k for k in x]

SWITZERLAND noted that the new criteria seek to simplify the incremental cost principle and the US welcomed the differentiation in criteria applied to the General Trust Fund and SCCF.

The REPUBLIC OF KOREA offered to host a global workshop. MOZAMBIQUE requested that a workshop be held in her country.


MICRONESIA emphasized SIDS� diffi culties in accessing\nfunds. NEW ZEALAND suggested that the GEF should be more\nfl exible and provide support for small-scale projects in SIDS.

